## LCEL - LangChain Expression Language
+ makes it easy to build complex chains from basic components
+ supports streaming, parallelism and logging

# Basic example: prompt + model + output parser

In [6]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('tell me a fun but important fact about {topic}')
model = ChatOpenAI(model='gpt-4')
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({'topic':'Cotuit, Massachusetts'})

'Cotuit, Massachusetts is known for its world-famous oysters. Cotuit Oysters have been harvested and enjoyed since the early 1800s and are internationally recognized as a high-quality seafood delicacy.'

## 1. Prompt
+ `prompt` is a `BasePromptTemplate`, it takes in a dictionary of template variables and produces a `PromptValue`
+ A `PromptValue` is a wrapper around a completed prompt that can be passed to an `LLM` or a `ChatModel`

In [11]:
prompt_value = prompt.invoke({'topic':'Oyster Harbors, Osterville, Massachusetts'})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a fun but important fact about Oyster Harbors, Osterville, Massachusetts')])

In [8]:
prompt_value.to_messages()

[HumanMessage(content='tell me a fun but important fact about Loop Beach, Cotuit, Massachusetts')]

In [9]:
prompt_value.to_string()

'Human: tell me a fun but important fact about Loop Beach, Cotuit, Massachusetts'

## 2. Model
+ The `PromptValue` is passed to `model`

In [12]:
message = model.invoke(prompt_value)
message

AIMessage(content="Oyster Harbors is a prestigious, private island community in Osterville, Massachusetts that was once a hunting retreat for wealthy Bostonians in the early 20th century. A fun but important fact is that it's connected to the mainland by a single, guarded bridge, ensuring the privacy and security of its residents.")

+ If the `model` was an `LLM`, it would outpit a string

In [14]:
from langchain_openai.llms import OpenAI

llm = OpenAI(model='gpt-3.5-turbo-instruct')
llm.invoke(prompt_value)

'\n\nAI: Oyster Harbors in Osterville, Massachusetts is home to the oldest and one of the most prestigious private golf clubs in the United States, the Oyster Harbors Club. It was founded in 1926 and has hosted several U.S. Open qualifiers, making it a must-visit destination for golf enthusiasts.'

## 3. Output parser
+ Pass the `model` output the the `output_parser`

In [15]:
output_parser.invoke(message)

"Oyster Harbors is a prestigious, private island community in Osterville, Massachusetts that was once a hunting retreat for wealthy Bostonians in the early 20th century. A fun but important fact is that it's connected to the mainland by a single, guarded bridge, ensuring the privacy and security of its residents."

## 4. Entire Pipeline
+ 1. Pass in user input on desired topics as `{'topic':'Cotuit, Massachusetts'}`
  2. The `prompt` component takes the user input, which constructs a PromptValue using the `topic`
  3. The `model` component takes the prompt and passes to the OpenAI LLM model; the output is a `ChatMessage` object
  4. The `output_parser` components takes in a `ChatMessage` object and transforms to a python string

In [16]:
input = {'topic':'Cotuit, Massachusetts'}
prompt.invoke(input)
(prompt | model).invoke(input)

AIMessage(content='Cotuit, Massachusetts is known for its cultivation of oysters. The Cotuit Oyster Company has been farming Cotuit Oysters since 1837, making it one of the oldest companies in the United States.')

# RAG Search Example

In [18]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatMessagePromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ['Brendan lives in Cotuit', 'Paris is the capital of France'],
    embedding=OpenAIEmbeddings()
)

retriver = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {'context': retriver, 'question': RunnablePassthrough()}
)

chain = setup_and_retrieval | prompt | model | output_parser
chain.invoke("Where does Brendan live?")

/Users/brendan/Desktop - Brendan’s MacBook Air/llm/lib/python3.10/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


ValidationError: 2 validation errors for DocArrayDoc
text
  Field required [type=missing, input_value={'embedding': [0.01788950... -0.005236115203535868]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing
metadata
  Field required [type=missing, input_value={'embedding': [0.01788950... -0.005236115203535868]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing